In case you want to learn how ETL is done, please run the following notebook first and update the file name below accordingly

https://github.com/IBM/coursera/blob/master/coursera_ml/a2_w1_s3_ETL.ipynb


In [2]:
# delete files from previous runs
!rm -f hmp.parquet*

# download the file containing the data in PARQUET format
!wget https://github.com/IBM/coursera/raw/master/hmp.parquet
    
# create a dataframe out of it
df = spark.read.parquet('hmp.parquet')

# register a corresponding query table
df.createOrReplaceTempView('df')

--2020-02-27 11:39:02--  https://github.com/IBM/coursera/raw/master/hmp.parquet
Resolving github.com (github.com)... 192.30.253.113
Connecting to github.com (github.com)|192.30.253.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/IBM/coursera/master/hmp.parquet [following]
--2020-02-27 11:39:02--  https://raw.githubusercontent.com/IBM/coursera/master/hmp.parquet
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 932997 (911K) [application/octet-stream]
Saving to: 'hmp.parquet'

100%[======================================>] 932,997     --.-K/s   in 0.05s   

2020-02-27 11:39:03 (18.5 MB/s) - 'hmp.parquet' saved [932997/932997]



In [3]:
df_energy = spark.sql("""
select sqrt(sum(x*x)+sum(y*y)+sum(z*z)) as label, class from df group by class
""")      
df_energy.createOrReplaceTempView('df_energy')          

In [4]:
df_join = spark.sql('select * from df inner join df_energy on df.class=df_energy.class')

In [5]:
df_join.show()

+---+---+---+--------------------+-----------+-----------------+-----------+
|  x|  y|  z|              source|      class|            label|      class|
+---+---+---+--------------------+-----------+-----------------+-----------+
| 22| 49| 35|Accelerometer-201...|Brush_teeth|11785.39634462923|Brush_teeth|
| 22| 49| 35|Accelerometer-201...|Brush_teeth|11785.39634462923|Brush_teeth|
| 22| 52| 35|Accelerometer-201...|Brush_teeth|11785.39634462923|Brush_teeth|
| 22| 52| 35|Accelerometer-201...|Brush_teeth|11785.39634462923|Brush_teeth|
| 21| 52| 34|Accelerometer-201...|Brush_teeth|11785.39634462923|Brush_teeth|
| 22| 51| 34|Accelerometer-201...|Brush_teeth|11785.39634462923|Brush_teeth|
| 20| 50| 35|Accelerometer-201...|Brush_teeth|11785.39634462923|Brush_teeth|
| 22| 52| 34|Accelerometer-201...|Brush_teeth|11785.39634462923|Brush_teeth|
| 22| 50| 34|Accelerometer-201...|Brush_teeth|11785.39634462923|Brush_teeth|
| 22| 51| 35|Accelerometer-201...|Brush_teeth|11785.39634462923|Brush_teeth|

In [6]:

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import Normalizer


vectorAssembler = VectorAssembler(inputCols=["x","y","z"],
                                  outputCol="features")
normalizer = Normalizer(inputCol="features", outputCol="features_norm", p=1.0)



In [7]:
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)



In [8]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[vectorAssembler, normalizer,lr])


In [9]:
model = pipeline.fit(df_join)


In [10]:
prediction = model.transform(df_join)


In [11]:
prediction.show()

+---+---+---+--------------------+-----------+-----------------+-----------+----------------+--------------------+------------------+
|  x|  y|  z|              source|      class|            label|      class|        features|       features_norm|        prediction|
+---+---+---+--------------------+-----------+-----------------+-----------+----------------+--------------------+------------------+
| 22| 49| 35|Accelerometer-201...|Brush_teeth|11785.39634462923|Brush_teeth|[22.0,49.0,35.0]|[0.20754716981132...|12586.729735016828|
| 22| 49| 35|Accelerometer-201...|Brush_teeth|11785.39634462923|Brush_teeth|[22.0,49.0,35.0]|[0.20754716981132...|12586.729735016828|
| 22| 52| 35|Accelerometer-201...|Brush_teeth|11785.39634462923|Brush_teeth|[22.0,52.0,35.0]|[0.20183486238532...|12542.703337345756|
| 22| 52| 35|Accelerometer-201...|Brush_teeth|11785.39634462923|Brush_teeth|[22.0,52.0,35.0]|[0.20183486238532...|12542.703337345756|
| 21| 52| 34|Accelerometer-201...|Brush_teeth|11785.3963446292

In [12]:
model.stages[2].summary.r2


0.03259100556263628